In [1]:
import os
import re
from urllib.parse import unquote

# --- CONFIGURATION ---
ROOT_DIR = '../temp3' 
ASSETS_DIR_NAME = 'assets'

# --- SMART REGEX EXPLANATION ---
# 1. Matches the opening parenthesis '(' of the markdown link.
# 2. Looks for 'assets/'.
# 3. Captures the filename even if it has nested parentheses like 'Pict(09).jpg'.
#    It does this by matching either "non-parenthesis characters" OR "characters inside parentheses".
LINK_PATTERN = re.compile(r'\((.*?assets/((?:[^)\n]|\([^)\n]*\))+))\)')

def find_file_on_disk(root_folder, relative_path):
    """
    Tries 4 different ways to find the file, handling URL encoding and + signs.
    Returns: (Absolute Path found, Boolean Success)
    """
    # Remove query parameters (e.g., image.jpg?format=1500w)
    clean_rel_path = relative_path.split('?')[0]
    
    # 1. Try EXACT match (Literal +)
    path_literal = os.path.normpath(os.path.join(root_folder, clean_rel_path))
    if os.path.exists(path_literal):
        return path_literal, True

    # 2. Try URL DECODED (e.g., %20 -> Space)
    path_decoded = os.path.normpath(os.path.join(root_folder, unquote(clean_rel_path)))
    if os.path.exists(path_decoded):
        return path_decoded, True

    # 3. Try PLUS to SPACE (e.g., SYR+SPEI -> SYR SPEI)
    path_space = os.path.normpath(os.path.join(root_folder, clean_rel_path.replace('+', ' ')))
    if os.path.exists(path_space):
        return path_space, True

    # 4. Try DECODED + PLUS to SPACE
    path_hybrid = os.path.normpath(os.path.join(root_folder, unquote(clean_rel_path).replace('+', ' ')))
    if os.path.exists(path_hybrid):
        return path_hybrid, True

    return path_literal, False

def rename_and_update():
    print(f"--- STARTING FINAL CLEANUP in: {os.path.abspath(ROOT_DIR)} ---\n")
    
    files_processed = 0
    assets_renamed = 0
    missing_log = []

    for root, dirs, files in os.walk(ROOT_DIR):
        if ASSETS_DIR_NAME in root: continue

        for file in files:
            if not file.endswith('.qmd'): continue

            qmd_path = os.path.join(root, file)
            qmd_name = os.path.splitext(file)[0] # e.g., "20230101-blog-title"
            
            with open(qmd_path, 'r', encoding='utf-8') as f:
                content = f.read()

            matches = LINK_PATTERN.findall(content)
            if not matches: continue

            print(f"Scanning: {file}")
            
            # Sort and unique to handle duplicates gracefully
            unique_links = sorted(list(set(matches)))
            
            new_content = content
            changes_made = False
            counter = 1

            for full_link_in_md, filename_part in unique_links:
                
                # 'full_link_in_md' is like "../assets/Pict+(09).jpg?format=original"
                # 'filename_part' is like "Pict+(09).jpg?format=original"

                # 1. FIND THE FILE
                current_abs_path, found = find_file_on_disk(root, full_link_in_md)
                
                if not found:
                    print(f"  [MISSING] {filename_part}")
                    missing_log.append(filename_part)
                    continue

                # 2. DETERMINE EXTENSION (Get it from the real file on disk)
                _, ext = os.path.splitext(current_abs_path)
                if not ext: _, ext = os.path.splitext(filename_part.split('?')[0])
                if not ext: ext = ".jpg" # Last resort fallback

                # 3. GENERATE NEW NAME (qmdname-01.ext)
                new_filename = f"{qmd_name}-{counter:02d}{ext}"
                counter += 1

                assets_dir = os.path.dirname(current_abs_path)
                new_abs_path = os.path.join(assets_dir, new_filename)

                # 4. RENAME FILE (If not already renamed)
                if current_abs_path != new_abs_path:
                    # Handle collisions (if 01 exists, bump to 02)
                    temp_counter = counter
                    while os.path.exists(new_abs_path):
                        # Use the same base logic but bump counter
                        new_filename = f"{qmd_name}-{temp_counter:02d}{ext}"
                        new_abs_path = os.path.join(assets_dir, new_filename)
                        temp_counter += 1

                    try:
                        os.rename(current_abs_path, new_abs_path)
                        assets_renamed += 1
                        print(f"  -> Renamed: {os.path.basename(current_abs_path)} => {new_filename}")
                    except OSError as e:
                        print(f"  [ERROR] Rename failed: {e}")
                        continue
                
                # 5. UPDATE CONTENT
                # We replace the *entire* captured link (path + filename + query params) 
                # with the new clean filename.
                new_rel_link = full_link_in_md.replace(filename_part, new_filename)
                
                # Remove query params from the link in the QMD too
                new_rel_link = new_rel_link.split('?')[0]
                
                new_content = new_content.replace(full_link_in_md, new_rel_link)
                changes_made = True

            if changes_made:
                with open(qmd_path, 'w', encoding='utf-8') as f:
                    f.write(new_content)
                files_processed += 1

    print(f"\n--- DONE ---")
    print(f"QMD Files Updated: {files_processed}")
    print(f"Assets Renamed:    {assets_renamed}")
    if missing_log:
        print(f"Files still missing: {len(missing_log)} (See console output)")

if __name__ == "__main__":
    rename_and_update()

--- STARTING FINAL CLEANUP in: C:\Users\benny\OneDrive\Documents\Github\site\temp ---

Scanning: index.qmd
  [MISSING] index-01.jpg
Scanning: 20031016-menduga-distribusi-ukuran-butir-hujan.qmd
  -> Renamed: butirhujan.jpg => 20031016-menduga-distribusi-ukuran-butir-hujan-01.jpg
  -> Renamed: kalibrasi.jpg => 20031016-menduga-distribusi-ukuran-butir-hujan-02.jpg
Scanning: 20031028-pengukuran-suhu.qmd
  -> Renamed: wet-bulb.jpg => 20031028-pengukuran-suhu-01.jpg
Scanning: 20031120-makalah-dasar-agronomi.qmd
  -> Renamed: Frankliniella_occidentalis_5364132-LGPT.jpg => 20031120-makalah-dasar-agronomi-01.jpg
Scanning: 20031125-mekanika-gerak-dalam-dua-dimensi.qmd
  -> Renamed: Gerak+dalam+Dua+Dimensi+2_Page_01.jpg => 20031125-mekanika-gerak-dalam-dua-dimensi-01.jpg
  -> Renamed: Gerak+dalam+Dua+Dimensi+2_Page_02.jpg => 20031125-mekanika-gerak-dalam-dua-dimensi-02.jpg
  -> Renamed: Gerak+dalam+Dua+Dimensi+2_Page_03.jpg => 20031125-mekanika-gerak-dalam-dua-dimensi-03.jpg
  -> Renamed: Gerak+d

In [1]:
import os
import re
import csv
from urllib.parse import urlparse

SITE_DIR    = r"C:\Users\benny\OneDrive\Documents\Github\site\temp3"
ASSETS_DIR  = os.path.join(SITE_DIR, "assets")
SUCCESS_CSV = os.path.join(ASSETS_DIR, "_download_success.csv")

def asset_key(u: str) -> str:
    p = urlparse(u)
    return f"{p.netloc.lower()}{p.path}"

def rewrite_qmd_links_from_success_csv(site_dir: str, assets_dir: str, success_csv: str):
    # 1) Build mapping host+path -> local filename
    key_to_local = {}
    with open(success_csv, "r", encoding="utf-8") as f:
        reader = csv.DictReader(f)
        for row in reader:
            local = row["local_filename"].strip()
            for col in ("requested_url", "final_url"):
                u = (row.get(col) or "").strip()
                if u:
                    key_to_local[asset_key(u)] = local

    # 2) Find any absolute URL (we’ll only replace those present in mapping)
    url_re = re.compile(r"https?://[^\s\"\'\)\]]+", re.IGNORECASE)

    updated_files = 0
    total_replaced = 0

    for root, _, files in os.walk(site_dir):
        # skip assets folder
        if os.path.abspath(root).startswith(os.path.abspath(assets_dir)):
            continue

        for fn in files:
            if not fn.endswith(".qmd"):
                continue

            fp = os.path.join(root, fn)
            with open(fp, "r", encoding="utf-8") as f:
                text = f.read()

            qmd_dir = os.path.dirname(fp)
            rel_assets = os.path.relpath(assets_dir, qmd_dir).replace("\\", "/")

            changed = False

            def repl(m):
                nonlocal changed, total_replaced
                u = m.group(0)
                p = urlparse(u)
                k = f"{p.netloc.lower()}{p.path}"
                if k in key_to_local:
                    changed = True
                    total_replaced += 1
                    return f"{rel_assets}/{key_to_local[k]}"
                return u  # leave untouched

            new_text = url_re.sub(repl, text)

            if changed and new_text != text:
                with open(fp, "w", encoding="utf-8", newline="\n") as f:
                    f.write(new_text)
                updated_files += 1

    print(f"Done. QMD files updated: {updated_files}")
    print(f"Total link replacements: {total_replaced}")

rewrite_qmd_links_from_success_csv(SITE_DIR, ASSETS_DIR, SUCCESS_CSV)


Done. QMD files updated: 1
Total link replacements: 1


In [2]:
import os
import re
import shutil
from collections import defaultdict
from urllib.parse import unquote

ROOT_DIR = r"C:\Users\benny\OneDrive\Documents\Github\site\temp3"
ASSETS_DIR_NAME = "assets"
COPY_SHARED_ASSETS = True   # True = copy shared assets per page, False = skip shared assets

# Matches links that contain assets/... inside markdown parentheses
MD_ASSET_LINK_RE = re.compile(r'\(([^)\n]*assets/[^)\n]+)\)')

# Also catch HTML attributes like src="...assets/..."
HTML_ASSET_ATTR_RE = re.compile(r'''(?:src|href)\s*=\s*["']([^"']*assets/[^"']+)["']''', re.IGNORECASE)

def find_file_on_disk(qmd_dir, link_path):
    """
    link_path: "../assets/file name.png" (may include query)
    Returns absolute path if exists.
    """
    clean = link_path.split("?")[0]

    candidates = [
        os.path.normpath(os.path.join(qmd_dir, clean)),
        os.path.normpath(os.path.join(qmd_dir, unquote(clean))),
        os.path.normpath(os.path.join(qmd_dir, clean.replace("+", " "))),
        os.path.normpath(os.path.join(qmd_dir, unquote(clean).replace("+", " "))),
    ]
    for p in candidates:
        if os.path.exists(p):
            return p
    return None

def extract_asset_links_from_qmd(text):
    links = []
    for m in MD_ASSET_LINK_RE.finditer(text):
        links.append(m.group(1))
    for m in HTML_ASSET_ATTR_RE.finditer(text):
        links.append(m.group(1))
    # preserve order but dedupe
    seen = set()
    out = []
    for x in links:
        if x not in seen:
            seen.add(x)
            out.append(x)
    return out

def safe_rename_assets(root_dir):
    assets_dir = os.path.join(root_dir, ASSETS_DIR_NAME)
    if not os.path.isdir(assets_dir):
        raise FileNotFoundError(f"Assets folder not found: {assets_dir}")

    # 1) Scan all QMDs and build references: abs_asset_path -> list[(qmd_path, link_path)]
    refs = defaultdict(list)
    qmd_files = []

    for r, _, files in os.walk(root_dir):
        if os.path.abspath(r).startswith(os.path.abspath(assets_dir)):
            continue
        for fn in files:
            if fn.endswith(".qmd"):
                qmd_files.append(os.path.join(r, fn))

    for qmd_path in qmd_files:
        qmd_dir = os.path.dirname(qmd_path)
        with open(qmd_path, "r", encoding="utf-8") as f:
            text = f.read()

        links = extract_asset_links_from_qmd(text)
        for link in links:
            abs_path = find_file_on_disk(qmd_dir, link)
            if abs_path:
                refs[abs_path].append((qmd_path, link))

    print(f"Found {len(refs)} unique asset files referenced by QMD.")
    shared = [p for p, lst in refs.items() if len(lst) > 1]
    print(f"Shared assets (referenced by >1 QMD): {len(shared)}")

    # 2) Process per QMD: assign sequential numbering per file
    per_qmd_counter = defaultdict(int)

    updated_qmd = 0
    renamed = 0
    copied = 0
    skipped_shared = 0

    # Preload all QMD contents
    qmd_texts = {}
    for qmd_path in qmd_files:
        with open(qmd_path, "r", encoding="utf-8") as f:
            qmd_texts[qmd_path] = f.read()

    # 3) Apply rename/copy and update all references
    for abs_asset_path, occurrences in refs.items():
        ext = os.path.splitext(abs_asset_path)[1] or ".jpg"
        is_shared = len(occurrences) > 1

        for (qmd_path, original_link) in occurrences:
            qmd_stem = os.path.splitext(os.path.basename(qmd_path))[0]
            per_qmd_counter[qmd_path] += 1
            idx = per_qmd_counter[qmd_path]

            new_filename = f"{qmd_stem}-{idx:02d}{ext}"
            new_abs_path = os.path.join(assets_dir, new_filename)

            # avoid collisions
            bump = idx
            while os.path.exists(new_abs_path):
                bump += 1
                new_filename = f"{qmd_stem}-{bump:02d}{ext}"
                new_abs_path = os.path.join(assets_dir, new_filename)

            if is_shared:
                if COPY_SHARED_ASSETS:
                    shutil.copy2(abs_asset_path, new_abs_path)
                    copied += 1
                else:
                    skipped_shared += 1
                    continue
            else:
                # unique: rename original file
                if abs_asset_path != new_abs_path:
                    os.rename(abs_asset_path, new_abs_path)
                    renamed += 1
                    # update key in refs so we don't try to rename again if something else hits it (rare)
                    abs_asset_path = new_abs_path

            # update link in QMD text: keep path prefix, replace filename, remove query
            clean_link = original_link.split("?")[0]
            prefix = clean_link.rsplit("/", 1)[0]
            new_link = f"{prefix}/{new_filename}"

            text = qmd_texts[qmd_path]
            if original_link in text:
                qmd_texts[qmd_path] = text.replace(original_link, new_link)
            elif clean_link in text:
                qmd_texts[qmd_path] = text.replace(clean_link, new_link)

    # 4) Write updated QMDs
    for qmd_path, new_text in qmd_texts.items():
        with open(qmd_path, "w", encoding="utf-8", newline="\n") as f:
            f.write(new_text)
        updated_qmd += 1

    print("\n--- DONE ---")
    print(f"QMD files rewritten: {updated_qmd}")
    print(f"Assets renamed (unique): {renamed}")
    print(f"Assets copied (shared):  {copied}")
    print(f"Shared skipped:          {skipped_shared} (COPY_SHARED_ASSETS=False)")

safe_rename_assets(ROOT_DIR)


Found 980 unique asset files referenced by QMD.
Shared assets (referenced by >1 QMD): 0

--- DONE ---
QMD files rewritten: 139
Assets renamed (unique): 980
Assets copied (shared):  0
Shared skipped:          0 (COPY_SHARED_ASSETS=False)
